In [ ]:
import numpy as np
import pandas as pd
import pulp
import plotly.express as px
import plotly.graph_objects as go


# Introduction to PuLP

In this demo notebook, we’ll take a quick look at [PuLP](https://pypi.org/project/PuLP/), a Python library for defining and solving linear programming (LP) problems. Our app in Chapter 8 uses PuLP to determine which warehouses a company should choose to reduce cost or carbon emissions. 

You don't need to be an optimization expert to follow along! This is just a lightweight intro so you can understand what’s happening under the hood when our app uses PuLP to solve problems. We'll be explaining the algorithms from our app, you can find more examples in [PuLP's GitHub repo / documentation](https://github.com/coin-or/pulp).

## What is Linear Programming?

Linear Programming is a mathematical method used to determine the best outcome (like maximum profit or lowest cost) in a situation with certain constraints (like limited resources, time, or capacity). It’s widely used in fields like logistics, finance, energy, and operations research.

## What is PuLP?
PuLP is a Python library that lets you:

* Define optimization problems using Python code
* Set constraints and objective functions
* Use solvers (like CBC, which comes with PuLP) to find optimal solutions. I also provides an API to use commercial solvers such as CPLEX, GUROBI and more

It’s a great tool for integrating optimization into Python applications, just like our demo app!


## Application Problem

Our app tries to solve a **facility location problem** using linear programming.

The first step of the application DOES NOT involve Linear Programming: 

1. First, we need to read in the data from our CSV files
2. Then we calculate a distance matrix between customers and warehouses using the [haversine formumla](https://en.wikipedia.org/wiki/Haversine_formula) (we create a function)

In [ ]:
df_warehouses = pd.read_csv("../src/data/warehouses.csv")
df_warehouses.head(2)


In [ ]:
df_customers = pd.read_csv("../src/data/customers.csv")
df_customers.head(2)


### Plotting the warehouses

This function plots the warehouses in a map, it's the one we use in the chapter (we add two helper functions to make it more modular):

In [ ]:
def _prepare_data(df_customers, df_warehouses):
    df_cust = df_customers.copy()
    df_wh = df_warehouses.copy()
    df_cust["type"] = "Customer"
    df_wh["type"] = "Warehouse"
    data = pd.concat([df_cust, df_wh])
    return data


def _add_columns_to_data(data):
    data["hover_amount"] = data.apply(
        lambda row: (
            f"Yearly Orders: {row['yearly_orders']}"
            if row["type"] == "Customer"
            else f"Yearly Rent: {row['yearly_cost']:,.0f} €"
        ),
        axis=1,
    )
    data["hover_name"] = data.apply(
        lambda row: (
            f"Company Name: {row['company_name']}"
            if row["type"] == "Customer"
            else f"Warehouse {row['warehouse']}"
        ),
        axis=1,
    )
    # Different marker size for Warehouses and customers
    data["size"] = data["type"].map({"Customer": 5, "Warehouse": 22})
    return data


def plot_customers_warehouses(df_customers, df_warehouses):
    data = _prepare_data(df_customers, df_warehouses)
    data = _add_columns_to_data(data)

    # Create the map
    fig = px.scatter_map(
        data,
        lat="latitude",
        lon="longitude",
        color="type",
        size="size",
        hover_name="hover_name",
        hover_data={
            "city": True,
            "hover_amount": True,
            "latitude": False,
            "longitude": False,
        },
        map_style="carto-positron",
        zoom=3,
        center={"lat": 50, "lon": 10},  # Center on Europe
        size_max=10,
    )

    fig.update_traces(
        hovertemplate="<b>%{hovertext}</b><br>%{customdata[0]}<br>%{customdata[1]}<extra></extra>"
    )
    # Customize legend (inside map, white background)
    fig.update_layout(
        title="📍 Customers & Potential Warehouses",
        height=700,
        margin={"r": 0, "t": 50, "l": 0, "b": 0},
        legend=dict(
            x=0.02,
            y=0.98,  # Position: top-left inside the map
            bgcolor="rgba(255,255,255,0.8)",  # White semi-transparent background
            bordercolor="black",
            borderwidth=1,
        ),
    )

    return fig


In [ ]:
plot_customers_warehouses(df_customers, df_warehouses)

Next, we create the functions to create the distance matrix:

In [ ]:
def haversine_vectorized(lat1, lon1, lat2, lon2):
    lat1 = np.radians(lat1[:, np.newaxis])  # shape (n, 1)
    lon1 = np.radians(lon1[:, np.newaxis])  # shape (n, 1)
    lat2 = np.radians(lat2[np.newaxis, :])  # shape (1, m)
    lon2 = np.radians(lon2[np.newaxis, :])  # shape (1, m)

    dlat = lat2 - lat1  # shape (n, m)
    dlon = lon2 - lon1  # shape (n, m)

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))

    R = 6371.0  # Radius of Earth in kilometers
    return R * c  # shape (n, m)

def calculate_distance_matrix(df_warehouses, df_customers):
    # Coordinates as np.arrays
    lat_wh = df_warehouses["latitude"].to_numpy()
    lon_wh = df_warehouses["longitude"].to_numpy()
    lat_cu = df_customers["latitude"].to_numpy()
    lon_cu = df_customers["longitude"].to_numpy()

    # Compute distance matrix using vectorized haversine
    distance_matrix = haversine_vectorized(lat_wh, lon_wh, lat_cu, lon_cu)

    return pd.DataFrame(
        distance_matrix,
        index=df_warehouses.index,
        columns=df_customers.index
    )

We calculate the distance matrix before we can start looking for optimum solutions:

In [ ]:
df_matrix = calculate_distance_matrix(df_warehouses, df_customers)
df_matrix.head()

## Optimization Time 🧮: Selecting the Best Warehouses

Now that we have our data and distance matrix ready, it's time to solve the actual optimization problem using PuLP.

Our objective is to minimize either the total cost or the CO₂ emissions of serving all customers from a subset of warehouses. We will:

1. Create an optimization model using PuLP
2. Define decision variables, the objective function, and constraints
3. Extract the optimal solution and present it as user-friendly data

To keep things clean and modular, we’ll break the process into smaller, well-defined functions before combining them in the main create_pulp_model function.

### Steps: 

1. Problem Initialization: return the `pulp.LpProblem()` object, setting the optimization goal. in our case, it's minimization (we want to minimie cost)
2. Defining Decision Variables. These variables represent:
    * Whether each warehouse is selected
    * Which warehouse each customer is assigned to
3. Setting the objective function: This function sets the optimization goal — minimize total cost or total CO₂.
4. Adding Constraints: These constraints make sure that:
    * Every customer is served
    * Only open warehouses serve customers
    * We respect the warehouse number and geographical rules
5. Running the optimization function --> This step happens in the main `create_pulp_model` function
6. Extract the results into pandas DataFrames


### Step 1 — Initialize the Optimization Problem

Before we can add variables, objectives, or constraints, we need to define our optimization problem.

We’ll create a minimization problem, which tells PuLP that our goal is to minimize something — in our case, total cost or total CO₂ emissions.

In [ ]:
def initialize_problem():
    """
    Initialize a PuLP minimization problem.
    """
    return pulp.LpProblem("Warehouse_Selection", pulp.LpMinimize)

### Step 2: Define Decision Variables
Now that we have an optimization problem initialized, we need to define what decisions the solver can make.

We define two sets of binary decision variables:

* `warehouse_var[w]`:
Equals 1 if warehouse w is opened, 0 otherwise.

* `assignment_var[(w, c)]`:
Equals 1 if customer c is assigned to warehouse w, 0 otherwise.

These are created using `pulp.LpVariable.dicts`, which lets us create many variables at once in a dictionary-like structure.

In [ ]:
def define_variables(df_warehouses, df_customers):
    """
    Creates binary decision variables for warehouse selection and customer assignment.
    """
    warehouses = df_warehouses.index.tolist()
    customers = df_customers.index.tolist()

    warehouse_var = pulp.LpVariable.dicts("Warehouse", warehouses, cat="Binary")
    assignment_var = pulp.LpVariable.dicts(
        "Assignment", [(w, c) for w in warehouses for c in customers], cat="Binary"
    )
    return warehouse_var, assignment_var

--> **What does `LpVariable.dicts` do?**
It creates a dictionary of variables.

`cat="Binary"` tells PuLP these variables can only take values 0 or 1 (typical for yes/no decisions in optimization).

You can access a variable like `warehouse_var[2]` or `assignment_var[(1, 5)]`. 

**For example:**

In [ ]:
define_variables(df_warehouses, df_customers)

### Step 3: Setting the objective function

Once the decision variables are in place, we need to tell the solver what we want to minimize.

Our app supports two optimization goals:

* Minimize cost (including warehouse rent + transportation)
* Minimize CO₂ emissions (including warehouse emissions + transport emissions)

To keep things modular and readable, we use two helper functions (prefixed with _) to build each version of the objective.


`pulp.lpSum(...)` is PuLP’s way of building up a linear expression: it sums over large sets of variables.

In both objectives, we combine fixed costs/emissions of operating a warehouse with the variable transport costs/emissions based on how far customers are from selected warehouses.

`prob +=` expression sets this expression as the thing we want to minimize; the `prob` variable comes from the initialization, it represents the problem we are trying to solve.

In [ ]:
def _objective_total(assignment_var, warehouse_var, df_warehouses, df_customers, distance_matrix,
                     transport_unit_cost, warehouse_cost_column, warehouse_cost_multiplier=1):
    """
    Generic objective builder for either price or CO₂.
    """
    warehouses = df_warehouses.index.tolist()
    customers = df_customers.index.tolist()

    transport_total = pulp.lpSum(
        assignment_var[(w, c)] *
        distance_matrix.at[w, c] *
        transport_unit_cost *
        df_customers.at[c, "yearly_orders"]
        for w in warehouses for c in customers
    )

    warehouse_total = pulp.lpSum(
        warehouse_var[w] *
        df_warehouses.at[w, warehouse_cost_column] *
        warehouse_cost_multiplier
        for w in warehouses
    )

    return transport_total + warehouse_total

In [ ]:
def set_objective_function(
    prob,
    assignment_var,
    warehouse_var,
    df_warehouses,
    df_customers,
    distance_matrix,
    optimize,
    price_per_km=4,
    co2_per_km=2
):
    """
    Adds the selected objective (price or co2) to the optimization problem.
    """
    if optimize == "price":
        prob += _objective_total(
            assignment_var,
            warehouse_var,
            df_warehouses,
            df_customers,
            distance_matrix,
            transport_unit_cost=price_per_km,
            warehouse_cost_column="yearly_cost"
        )
    else:
        prob += _objective_total(
            assignment_var,
            warehouse_var,
            df_warehouses,
            df_customers,
            distance_matrix,
            transport_unit_cost=co2_per_km,
            warehouse_cost_column="yearly_co2_tons",
            warehouse_cost_multiplier=1000  # Convert tons to kg
        )

### Step 4: Adding Constraints

This function tells the optimization model how the problem works. It defines the rules (called **constraints**) that the solution must follow. Without these rules, the model could give us crazy or meaningless results.

With PuLP, we build the problem by combining an objective function (from the function beofre this one!) with a list of constraints (what the solution is allowed to do). --> This function add the constraints to the same `prob` element. 

In other words: The objective + the constraints define a problem to solve.

#### What constraints we add, and WHY:

1. **Each customer must be assigned to exactly one warehouse**

Every customer needs to receive deliveries from some warehouse, but only one. 
This constraint forces the model to pick exactly one warehouse for each customer.

2. **Customers can only be assigned to warehouses that are open**

We don’t want the model to assign customers to warehouses we didn’t even open — that would be like shipping from an imaginary location.
This rule ties the assignment of a customer to a warehouse to whether that warehouse is actually selected/open.

3. **Limit the total number of warehouses**

The end application lets users select how many warehouses to add to the solution. Sometimes we want exactly `n` warehouses, sometimes we’re flexible.
This part adds either:

* A fixed number of warehouses (if the user says `number_of_warehouses` = `n`)
* Or a reasonable range (like between 1 and 10), if the user just says "any"

4. **(Optional) Require warehouses in specific countries**

Sometimes, for strategic, political, or logistical reasons, we want to make sure we operate in certain countries.
If the user passes a list like ["France", "Germany"], this rule tells the model:

"Hey, make sure you open at least one warehouse in France and one in Germany."

It’s optional — we only add this if the user asks for it.



In [ ]:
def add_constraints(prob, assignment_var, warehouse_var, df_warehouses, df_customers, number_of_warehouses, country_list=None):
    """
    Adds problem constraints.
    """
    warehouses = df_warehouses.index.tolist()
    customers = df_customers.index.tolist()

    # 1. Each customer is assigned to exactly one warehouse
    for c in customers:
        prob += pulp.lpSum(assignment_var[(w, c)] for w in warehouses) == 1

    # 2. Customers can only be assigned to selected warehouses
    for w in warehouses:
        for c in customers:
            prob += assignment_var[(w, c)] <= warehouse_var[w]

    # 3. Limit number of warehouses
    if number_of_warehouses != "any":
        prob += pulp.lpSum(warehouse_var[w] for w in warehouses) == int(number_of_warehouses)
    else:
        prob += pulp.lpSum(warehouse_var[w] for w in warehouses) >= 1
        prob += pulp.lpSum(warehouse_var[w] for w in warehouses) <= 10

    # 4. Country constraints (optional)
    if country_list:
        for country in country_list:
            wh_in_country = df_warehouses[df_warehouses["country"] == country].index.tolist()
            prob += pulp.lpSum(warehouse_var[w] for w in wh_in_country) >= 1


### Step 6. Extracting results

**Important!** You may think we forgot step 5... Not quite! Step 5 is to launch the optimization function, wich happens in the main function!

This step collects the results after the solver gives us a solution. It formats it so it’s human-readable and ready for analysis or visualization.

We're basically asking:

“Which warehouses did the model choose? Which customers are assigned to which warehouse? And what are the total costs and CO₂ emissions?”

In [ ]:
def extract_results(warehouse_var, assignment_var, df_warehouses, df_customers, distance_matrix, price_per_km=4, co2_per_km=2):
    """
    Extracts the results from the solved problem and formats them.
    """
    selected_warehouses = [w for w in df_warehouses.index if pulp.value(warehouse_var[w]) == 1]
    customers = df_customers.index.tolist()

    assignments = []
    scenario_costs = []
    scenario_co2s = []
    scenario_orders = []

    for w in selected_warehouses:
        yearly_cost = df_warehouses.at[w, "yearly_cost"]
        yearly_co2_tons = df_warehouses.at[w, "yearly_co2_tons"]
        total_transport_cost = 0
        total_transport_co2 = 0
        warehouse_orders = 0

        for c in customers:
            if pulp.value(assignment_var[(w, c)]) == 1:
                distance = distance_matrix.at[w, c]
                orders = df_customers.at[c, "yearly_orders"]

                total_transport_cost += distance * price_per_km * orders
                total_transport_co2 += distance * co2_per_km * orders
                warehouse_orders += orders

                assignments.append({
                    "warehouse": df_warehouses.at[w, "warehouse"],
                    "warehouse_lat": df_warehouses.at[w, "latitude"],
                    "warehouse_lon": df_warehouses.at[w, "longitude"],
                    "customer": df_customers.at[c, "company_name"],
                    "customer_lat": df_customers.at[c, "latitude"],
                    "customer_lon": df_customers.at[c, "longitude"],
                    "distance_km": distance,
                    "orders": orders,
                    "total_cost": int(distance * price_per_km * orders),
                    "total_co2_kg": int(distance * co2_per_km * orders),
                })

        scenario_costs.append(int(yearly_cost + total_transport_cost))
        scenario_co2s.append(int(yearly_co2_tons + total_transport_co2 / 1000))
        scenario_orders.append(warehouse_orders)

    df_selected = df_warehouses.loc[selected_warehouses].copy()
    df_selected["scenario_cost"] = scenario_costs
    df_selected["scenario_co2_tons"] = scenario_co2s
    df_selected["scenario_orders"] = scenario_orders
    df_assignments = pd.DataFrame(assignments)

    return df_selected, df_assignments

### Main optimization function

We call all our previous functions from a main module.

**To summarize:**

1. We create an empty problem (`prob` variable)
2. We create variables for the mathematical equations
3. We add the objective: this is what to solve
4. We add the constraints: this is the conditions the solution needs to respect
5. We solve the problem using a solver, in our case, we use `PULP_CBC_CMD`:**

```python
    prob.solve(pulp.PULP_CBC_CMD(msg=False))
```

6. We extract the results

In our case:

1. We create a minimization problem
2. Our variables are the warehouses that we can select, and each possible warehouse - customer combination
3. Our objective can be selected by users (from the app's UI). They have to choices:
    * Reduce (optimize) price (cost)
    * Reduce (optimize) CO2 emissions
4. Some constraints are "mandatory", like each customer needs to be asigned to one and only one warehouse. Some constraints are selected by the user from the UI (like mandatory country presence, or number of warehouses to include in the solution)
5. We use the `PULP_CBC_CMD` solver, because this is a simple problem.
6. We extract 2 DataFrames from the solution:
    * The warehouses from the solution
    * The customer - warehouse relationship

In [ ]:
def create_pulp_model(
    df_warehouses,
    df_customers,
    distance_matrix,
    optimize="price",
    number_of_warehouses="any",
    country_list=None,
    price_per_km=4,
    co2_per_km=2
):
    """
    Creates and solves the warehouse optimization problem using PuLP.
    """
    df_warehouses = df_warehouses.copy()
    df_customers = df_customers.copy()
    df_warehouses["id"] = df_warehouses.index
    df_customers["id"] = df_customers.index

    prob = initialize_problem()
    warehouse_var, assignment_var = define_variables(df_warehouses, df_customers)
    set_objective_function(prob, assignment_var, warehouse_var, df_warehouses, df_customers, distance_matrix, optimize, price_per_km, co2_per_km)
    add_constraints(prob, assignment_var, warehouse_var, df_warehouses, df_customers, number_of_warehouses, country_list)
    prob.solve(pulp.PULP_CBC_CMD(msg=False))

    return extract_results(warehouse_var, assignment_var, df_warehouses, df_customers, distance_matrix, price_per_km, co2_per_km)


## Using our function!

We create a notebook cell with variables, feel free to change their value and then re-run the model below!

In [ ]:
number_of_warehouses = 3
optimize = "price"  # or "co2"
country_list = ["France", "Germany"]  # optional
price_per_km = 4
co2_per_km = 2

In [ ]:
# Run optimization
df_selected, df_assignments = create_pulp_model(
    df_warehouses,
    df_customers,
    df_matrix,
    optimize=optimize,
    number_of_warehouses=number_of_warehouses,
    country_list=country_list,
    price_per_km=price_per_km,
    co2_per_km=co2_per_km
)

# Display results
df_selected.head()

In [ ]:
df_assignments.head()

## Plotting the results!

We create a different version of the map function to show the selected warehouses and the link o each customer:

In [ ]:
def _prepare_data(df_assignments):
    # Extract unique warehouses and customers
    df_warehouses_unique = df_assignments[
        ["warehouse", "warehouse_lat", "warehouse_lon"]
    ].drop_duplicates()
    df_warehouses_unique = df_warehouses_unique.rename(
        columns={"warehouse": "name", "warehouse_lat": "lat", "warehouse_lon": "lon"}
    )
    df_warehouses_unique["type"] = "Warehouse"

    df_customers_unique = df_assignments[
        ["customer", "customer_lat", "customer_lon"]
    ].drop_duplicates()
    df_customers_unique = df_customers_unique.rename(
        columns={"customer": "name", "customer_lat": "lat", "customer_lon": "lon"}
    )
    df_customers_unique["type"] = "Customer"

    data = pd.concat([df_warehouses_unique, df_customers_unique], ignore_index=True)
    return data


def _add_columns_to_data(data):
    # Prepare hover name and size
    data["hover_name"] = data.apply(
        lambda row: (
            f"Warehouse {row['name']}"
            if row["type"] == "Warehouse"
            else f"Customer: {row['name']}"
        ),
        axis=1,
    )
    data["size"] = data["type"].map({"Warehouse": 22, "Customer": 5})
    return data


def plot_assignments(df_assignments):
    data = _prepare_data(df_assignments)
    data = _add_columns_to_data(data)

    # Create the base figure with markers
    fig = px.scatter_map(
        data,
        lat="lat",
        lon="lon",
        color="type",
        color_discrete_map={"Customer": "blue", "Warehouse": "red"},
        size="size",
        hover_name="hover_name",
        hover_data={
            "lat": False,
            "lon": False,
            "type": False,
            "name": False,
            "size": False,
        },
        map_style="carto-positron",
        zoom=3,
        center={"lat": 50, "lon": 10},  # Center on Europe
        size_max=10,
    )

    # Prepare line coordinates for assignments
    lons = []
    lats = []
    for _, row in df_assignments.iterrows():
        lons.extend([row["warehouse_lon"], row["customer_lon"], None])
        lats.extend([row["warehouse_lat"], row["customer_lat"], None])

    # Add lines to the figure
    fig.add_trace(
        go.Scattermap(
            lon=lons,
            lat=lats,
            mode="lines",
            line=dict(width=1, color="#003399"),
            showlegend=False,
        )
    )

    # Customize layout similar to the example function
    fig.update_layout(
        title="📍 Customer-Warehouse Assignments",
        height=700,
        margin={"r": 0, "t": 50, "l": 0, "b": 0},
        legend=dict(
            x=0.02,
            y=0.98,
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="black",
            borderwidth=1,
        ),
    )

    # Update hovertemplate to only show the hover_name
    fig.update_traces(hovertemplate="<b>%{hovertext}</b><extra></extra>")

    return fig

In [ ]:
plot_assignments(df_assignments)